In [1]:
import os
import re
import sys
import uuid
import hashlib
import socket
import requests
import json
import time
from Crypto.PublicKey import RSA
from Crypto.Cipher import PKCS1_v1_5
from time import time
from urllib.parse import urlparse
from flask import request
from operator import itemgetter
from datetime import datetime, timezone
from pyp2p.net import *

from config import Config

In [2]:
uid = uuid.uuid4().hex
password = 'lalalend'
passHashed = hashlib.sha3_224((password+uid).encode()).hexdigest()
passHashed

'2f1d2c396fe1aeec398cee910d55c44f254e470ee7a927e4890bcd7a'

In [3]:
class Account(object):
    def __init__(self):
        self.slt = None

In [4]:
def createWallet(password, account):
    # Create wallet ID
    uid = uuid.uuid4().hex
    hsh = hashlib.sha3_224((password+uid).encode()).hexdigest()

    # Generate keys pair
    key = RSA.generate(1024)
    # Export public and private keys and save them
    pubKey = key.publickey().export_key(format='DER')
    with open(os.path.join(os.path.join(Config().BASEDIR, 'keys'), f'{hsh}_pubKey.der'), 'wb') as pubFile:
        pubFile.write(pubKey)

    prKey = key.export_key(format='DER', passphrase=password, pkcs=8,
                              protection="scryptAndAES128-CBC")
    with open(os.path.join(os.path.join(Config().BASEDIR, 'keys'), f'{hsh}_prKey.der'), 'wb') as prFile:
        prFile.write(prKey)
        
    account.slt = uid
        
    return f'{hsh}_pubKey.der', f'{hsh}_prKey.der'

In [5]:
acc = Account()
acc.slt

In [6]:
acc1 = Account()
acc1.slt

In [7]:
acc.slt = 'asd'
acc1.slt = 'zxc'

In [10]:
createWallet('12345', acc)

('01cc1b035eecce462b87cdfcb45ed91a71ea9827331945d0f2e44986_pubKey.der',
 '01cc1b035eecce462b87cdfcb45ed91a71ea9827331945d0f2e44986_prKey.der')

In [8]:
acc.slt

'asd'

In [11]:
createWallet('123', acc1)

('ceb0d109964dc56233551e82519d2fa37be874342833aeb1c385e607_pubKey.der',
 'ceb0d109964dc56233551e82519d2fa37be874342833aeb1c385e607_prKey.der')

In [9]:
acc1.slt

'zxc'

In [13]:
fileNames = os.listdir(os.path.join(Config().BASEDIR, 'keys'))
fileNames

['312e1b08ba1a4d6b8697c569035868a4_pubKey.der',
 '9ce57bfddcd3a64b4b104830a7c6e0b18ea73001cc3ebff0bf2088a9_pubKey.der',
 'da2a0b024ee53784553f3f700cf026c734a402df0a2e3aa7dabc8641_prKey.der',
 '9c26c9731ff186b9de8df41467fc8c9af29f05a7587c5283b907aa33_prKey.der',
 'c578c71ea8bea80c9afaf41396b7bc00add984f2484d8c32f45ccdb8_prKey.der',
 '.DS_Store',
 'c578c71ea8bea80c9afaf41396b7bc00add984f2484d8c32f45ccdb8_pubKey.der',
 '01cc1b035eecce462b87cdfcb45ed91a71ea9827331945d0f2e44986_pubKey.der',
 '0c7d6195ffac8d648a095643396f68df70e62bbcd3614ad4ea8d384a_prKey.der',
 '730b642be9fad40694f1bdd2a246640b4255f58d52d100a3c8c674b4_pubKey.der',
 '81943ae827bd0cb06a2462c68bf9f3f3cc41976c407b977274dbcfbf_pubKey.der',
 '9ce57bfddcd3a64b4b104830a7c6e0b18ea73001cc3ebff0bf2088a9_prKey.der',
 '5995b45c713a8f35acefcf9382262f6b530ba25913a791f74b1bb243_prKey.der',
 'ceb0d109964dc56233551e82519d2fa37be874342833aeb1c385e607_prKey.der',
 '01cc1b035eecce462b87cdfcb45ed91a71ea9827331945d0f2e44986_prKey.der',
 '5995b45c

In [12]:
password = '12345'
accounts = [acc, acc1]

In [15]:
fileNames = os.listdir(os.path.join(Config().BASEDIR, 'keys'))
password = '123'
accounts = [acc, acc1]
keysNames = []

for account in accounts:
    hsh = hashlib.sha3_224((password+account.slt).encode()).hexdigest()
    for name in fileNames:
        if name.split('_')[0] == hsh:
            print(name)
            keysNames.append(name)
    
#     hsh = hashlib.sha3_224((password+account.slt).encode()).hexdigest()
#     print(hsh)
#     keysNames = [name for name in fileNames if name.split('_')[0] == hsh]
                

# keysNames = [name for name in fileNames if name.split('_')[0] ==\
#                 [hashlib.sha3_224((password+account.slt).encode()).hexdigest()\
#                 for account in accounts][1]]

keysNames

ceb0d109964dc56233551e82519d2fa37be874342833aeb1c385e607_prKey.der
ceb0d109964dc56233551e82519d2fa37be874342833aeb1c385e607_pubKey.der


['ceb0d109964dc56233551e82519d2fa37be874342833aeb1c385e607_prKey.der',
 'ceb0d109964dc56233551e82519d2fa37be874342833aeb1c385e607_pubKey.der']

In [65]:
accounts[0].slt

'86abb765ecc04af2bd784e9e285dda62'

In [2]:
def proofExpenditure(balance, sendAmount):
    if balance - sendAmount >= 0:
        return True
    else:
        return False

In [8]:
balance = 1000
sendAmount = 1000.0000001
proofExpenditure(balance, sendAmount)

False

In [28]:
whiteIp = requests.get('https://api.ipify.org').content
whiteIp.decode()

'82.151.196.144'

In [27]:
requests.get('http://178.176.120.241:5000').content

b'{"MSG":"Working"}\n'

In [12]:
class Acc(object):
    def __init__(self):
        self.address = None
        self.balance = 1000
#         self.c = 0
        
    @staticmethod
    def count(c):
        c += 1
        return c

In [3]:
ac = Acc()

In [13]:
acc1 = Acc()
acc2 = Acc()

acc1.balance = 1
acc2.balance = 2

acc1.address = 'a'
acc2.address = 'b'

In [19]:
accounts = [acc1, acc2]
accounts

[<__main__.Acc at 0x7ff3c6308550>, <__main__.Acc at 0x7ff3c63084e0>]

In [21]:
for account in accounts:
    if account.address == 'a':
        a = account
a

In [26]:
sender = 'a'
account = [account for account in accounts if account.address == sender][0]
account

In [66]:
acBal = [acc.balance for acc in accounts if acc.address == 'b'][0]
acBal

2

In [106]:
def a(acc):
    return acc.balance

In [108]:
a(ac)

1000

In [104]:
a.count(2)

3

In [109]:
from blockchain import Blockchain

In [110]:
blockchain = Blockchain()

In [112]:
toHash = 'To hash'
type(blockchain.hash(toHash))

str

## Test Blockchain Endpoints

In [17]:
host = 'http://0.0.0.0:5000'

In [3]:
nodes = requests.get(host+"/nodes/getNodes").content
nodes

b'{"nodes": ["178.176.120.241:5000"]}'

In [277]:
host1 = 'http://178.176.120.241:5000'
nodes1 = requests.get(host1+"/nodes/getNodes").content
nodes1

b'{"nodes": ["82.151.196.144:5000"]}'

In [115]:
wallet = json.loads(wallet)['ADDRESS']
wallet

'z01x006cd240bb549be12078aa8cf76dac965060dd2b6ba2f4151b6b0fe680'

In [12]:
regNode = requests.post(host+"/nodes/register", json={"nodes": ['http://0.0.0.0:5001']}).content
regNode

In [69]:
# logout = requests.get(host+'/wallet/logout').content
# logout

b'{"MSG":true}\n'

In [224]:
# Coinbase
wallet = requests.post(host+'/wallet/new', json={'password': '12345'}).content
wallet

b'{"ADDRESS":"z01x0008df7439ba5eb0ef732617b2427ce41df9931aa915b09a0ea4496087"}\n'

In [234]:
wallet = requests.post(host+'/wallet/new', json={'password': 'qwerty'}).content
wallet

b'{"ADDRESS":"z01x00742174f4890b1eac3ecf4a8810adce6a3aecbf622fd0f43f45b82410"}\n'

In [225]:
login = requests.post(host+'/wallet/login', json={'password': '12345'}).content
login

b'{"ADDRESS":"z01x0008df7439ba5eb0ef732617b2427ce41df9931aa915b09a0ea4496087","MSG":true}\n'

In [230]:
mineReq = requests.get(host+'/mine').content
mineReq

b'{"index":4,"message":"New Block Forged","pool_syncing":"Tx pool synced among 0 nodes","previous_hash":"ca428fd492dca1d47a5fc95d16b46e9bf4aa42c0f4ac619ba4b43e4a64295b0d","proof":26333,"transactions":[{"comissionAmount":1.0,"contract":null,"price":0.0,"recieveAmount":null,"recipient":"z01x0008df7439ba5eb0ef732617b2427ce41df9931aa915b09a0ea4496087","sendAmount":1,"sender":"z01x0000000000000000000000000000000000000000000000000000000000","symbol":"zsh","timestamp":1630067598.575778,"tradeTxId":null}]}\n'

In [235]:
accountBalance = requests.post(host+'/wallet/getBalance', json={'address': json.loads(login)['ADDRESS']}).content
accountBalance

b'{"BALACNE":{"balance":1.0,"token":"ZSH"}}\n'

In [237]:
sellTx = {
        'type': 'trade',
        'sender': json.loads(login)['ADDRESS'],
        'symbol': 'zsh/usdt',
        'price': 9.93,
        'send': 'zsh',
        'sendVol': 1.0,
        'get': 'usdt',
        'getVol': 9.93,
        'comissionAmount':2
    }
txReq = requests.post(host+'/transactions/new', json=sellTx).content
txReq

b'{"MSG":"Spend amount exceeds account balance"}\n'

In [238]:
login = requests.post(host+'/wallet/login', json={'password': 'qwerty'}).content
login

b'{"ADDRESS":"z01x00742174f4890b1eac3ecf4a8810adce6a3aecbf622fd0f43f45b82410","MSG":true}\n'

In [239]:
accountBalance = requests.post(host+'/wallet/getBalance', json={'address': json.loads(login)['ADDRESS']}).content
accountBalance

b'{"BALACNE":{"balance":0.0,"token":"ZSH"}}\n'

In [240]:
buyTx = {
        'type': 'trade',
        'sender': json.loads(login)['ADDRESS'],
        'symbol': 'zsh/usdt',
        'price': 9.93,
        'send': 'usdt',
        'sendVol': 9.93,
        'get': 'zsh',
        'getVol': 1.0,
        'comissionAmount':2
    }
txReq = requests.post(host+'/transactions/new', json=buyTx).content

txReq

b'{"MSG":"Spend amount exceeds account balance"}\n'

In [206]:
mineReq = requests.get(host+'/mine').content
mineReq

b'{"index":5,"message":"New Block Forged","pool_syncing":"Tx pool synced among 0 nodes","previous_hash":"838b7acd3cc11c1d06a89beb74a9602df0b426922146f072a50ee6cb724b0731","proof":22850,"transactions":[{"comissionAmount":1.0,"contract":null,"price":0.0,"recieveAmount":null,"recipient":"z01x001a96a696dbd36fd45f45f1bbd37e8a2013b158f7f9033d77c8ec9f3d","sendAmount":1,"sender":"z01x0000000000000000000000000000000000000000000000000000000000","symbol":"zsh","timestamp":1630066637.382237,"tradeTxId":null},{"comissionAmount":0.02,"contract":"zsh","price":9.93,"recieveAmount":19.86,"recipient":"z01x00674a93e7d0d3a9d7634d0b613352454505bbb4313ec9fe74394ae7d3","sendAmount":2.0,"sender":"z01x001a96a696dbd36fd45f45f1bbd37e8a2013b158f7f9033d77c8ec9f3d","symbol":"zsh/usdt","timestamp":1630066638.299597,"tradeTxId":"367b1a9b0558287d68fe1d9f76ea304d3824d2caa28cd490973bbeea5bc5dc24"},{"comissionAmount":0.02,"contract":"usdt","price":9.93,"recieveAmount":2.0,"recipient":"z01x001a96a696dbd36fd45f45f1bbd37e8a

In [190]:
accountBalance = requests.post(host+'/wallet/getBalance', json={'address': json.loads(login)['ADDRESS']}).content
accountBalance

b'{"BALACNE":{"balance":2.0,"token":"ZSH"}}\n'

In [130]:
step=0.1
for i in range(3):
    
    tradeTx = {
        'type': 'trade',
        'sender': json.loads(login)['ADDRESS'],
        'symbol': 'zsh/usdt',
        'price': 10.0+step,
        'send': 'zsh',
        'sendVol': 1.0,
        'get': 'usdt',
        'getVol': 10.0+step,
        'comissionAmount':2
    }

    txReq = requests.post(host+'/transactions/new', json=tradeTx).content
    step += 0.1

txReq

b'{"MSG":4,"SYNC":"Tx pool synced among 1 nodes"}\n'

In [131]:
step = 0.1
for i in range(3):
    tradeTx = {
        'type': 'trade',
        'sender': json.loads(login)['ADDRESS'],
        'symbol': 'zsh/usdt',
        'price': 9.8+step,
        'send': 'usdt',
        'sendVol': 9.8+step,
        'get': 'zsh',
        'getVol': 1.0,
        'comissionAmount':2
    }
    
    txReq = requests.post(host+'/transactions/new', json=tradeTx).content
    step -= 0.1
txReq

b'{"MSG":4,"SYNC":"Tx pool synced among 1 nodes"}\n'

In [36]:
ordersPool = json.loads(requests.get(host+'/getTradeOrders').content)['tradeOrders']
ordersPool

[{'comissionAmount': 2.0,
  'get': 'usdt',
  'getVol': 10.1,
  'price': 10.1,
  'send': 'zsh',
  'sendVol': 1.0,
  'sender': 'z01x006cd240bb549be12078aa8cf76dac965060dd2b6ba2f4151b6b0fe680',
  'symbol': 'zsh/usdt',
  'timestamp': 1629980429.780089,
  'tradeTxId': 'cc9828d5acda1758fc2435ad279c436213bd5d65a6767e557d9b0f9c88794a9c'},
 {'comissionAmount': 2.0,
  'get': 'usdt',
  'getVol': 10.2,
  'price': 10.2,
  'send': 'zsh',
  'sendVol': 1.0,
  'sender': 'z01x006cd240bb549be12078aa8cf76dac965060dd2b6ba2f4151b6b0fe680',
  'symbol': 'zsh/usdt',
  'timestamp': 1629980430.591604,
  'tradeTxId': 'f82a51ea6f4e22fd10c4984a9b20965774e4f308eeae3d039dea659034c03d41'},
 {'comissionAmount': 2.0,
  'get': 'usdt',
  'getVol': 10.3,
  'price': 10.3,
  'send': 'zsh',
  'sendVol': 1.0,
  'sender': 'z01x006cd240bb549be12078aa8cf76dac965060dd2b6ba2f4151b6b0fe680',
  'symbol': 'zsh/usdt',
  'timestamp': 1629980431.33705,
  'tradeTxId': '040d642e897cf24f6784de7ccdc1700bcf8f56881aa1d80751f65c2da2105573'},
 {

In [132]:
chain = json.loads(requests.get(host+'/chain').content)['chain']
chain

[{'index': 1,
  'previousHash': '9c3b78de7eb0066418a682aec7d0e6841cd5cbe70d71ae65309d4917cd1c5681',
  'proof': 42,
  'timestamp': 1629032178.368946,
  'transactions': ['ascl,sdc,d[v,[d,vsdcaaxasx]]', 'lv,sldc,sdc ;,d;d']},
 {'index': 2,
  'previousHash': '9204ca3bc9ca2716f5a0a3ed866c9fb5b15a292c0ea53bc2e3e145b94933f393',
  'proof': 22976,
  'timestamp': 1630062487.100557,
  'transactions': [{'comissionAmount': 1.0,
    'contract': None,
    'price': 0.0,
    'recieveAmount': None,
    'recipient': 'z01x0028f3a682c732e293aad870ea7ed6bdd79b0fca340cf6927cf9e9c10a',
    'sendAmount': 1,
    'sender': 'z01x0000000000000000000000000000000000000000000000000000000000',
    'symbol': 'zsh',
    'timestamp': 1630062486.249603,
    'tradeTxId': None}]},
 {'index': 3,
  'previousHash': '422699de7eaf9da0437a4e67a69bbc18192eef6d4d145112fac4caaa8c72826b',
  'proof': 119240,
  'timestamp': 1630062491.103936,
  'transactions': [{'comissionAmount': 1.0,
    'contract': None,
    'price': 0.0,
    'recie

In [71]:
# 'comissionAmount': 2.0,
#   'get': 'usdt',
#   'getVol': 1030.0,
#   'price': 1030.0,
#   'send': 'uni',
#   'sendVol': 1.0,
#   'sender': 'z01x0030dfc2c4736a6ecaa2362cb65f7fb313934b33283f51caa765e3745e',
#   'symbol': 'uni/usdt',
#   'timestamp': 1629813262.264664,
#   'tradeTxId': 'd648507f2b334c965c3e616f6d06fb1e987e820453385b8e65183c2f994265c3'}


buyTx = {
        'type': 'trade',
        'sender': json.loads(login)['ADDRESS'],
        'symbol': 'uni/usdt',
        'price': 1000.0,
        'send': 'usdt',
        'sendVol': 1000.0,
        'get': 'uni',
        'getVol': 1.0,
        'comissionAmount':2
    }
txReq = requests.post(host+'/transactions/new', json=buyTx).content


# {'comissionAmount': 2.0,
#   'get': 'usdt',
#   'getVol': 49010.0,
#   'price': 49010.0,
#   'send': 'btc',
#   'sendVol': 1.0,
#   'sender': 'z01x0030dfc2c4736a6ecaa2362cb65f7fb313934b33283f51caa765e3745e',
#   'symbol': 'btc/usdt',
#   'timestamp': 1629813003.165863,
#   'tradeTxId': 'b0c04cc31ff140a01b7bacaad8dd8b866b93861edc74e4e2be28e0327e31c139'}

# buyTx = {
#         'type': 'trade',
#         'sender': json.loads(login)['ADDRESS'],
#         'symbol': 'btc/usdt',
#         'price': 49010.1,
#         'send': 'usdt',
#         'sendVol': 49010.1,
#         'get': 'btc',
#         'getVol': 1.0,
#         'comissionAmount':2
#     }
txReq = requests.post(host+'/transactions/new', json=buyTx).content
txReq

b'{"MSG":4,"SYNC":"Tx pool synced among 1 nodes"}\n'

In [48]:
ordersPool = json.loads(requests.get(host+'/getTradeOrders').content)['tradeOrders']
ordersPool

[{'comissionAmount': 2.0,
  'get': 'usdt',
  'getVol': 1010.0,
  'price': 1010.0,
  'send': 'uni',
  'sendVol': 1.0,
  'sender': 'z01x0030dfc2c4736a6ecaa2362cb65f7fb313934b33283f51caa765e3745e',
  'symbol': 'uni/usdt',
  'timestamp': 1629891726.53693,
  'tradeTxId': '562bea1978aa9504102791d5b466ea68e40cf62605659248170a5ed150e53857'},
 {'comissionAmount': 2.0,
  'get': 'usdt',
  'getVol': 1020.0,
  'price': 1020.0,
  'send': 'uni',
  'sendVol': 1.0,
  'sender': 'z01x0030dfc2c4736a6ecaa2362cb65f7fb313934b33283f51caa765e3745e',
  'symbol': 'uni/usdt',
  'timestamp': 1629891727.30612,
  'tradeTxId': 'cdbb497b923da5f1e8c473e2e4d74e0f78ff0b7897c728798c4c0b7ae36218c0'},
 {'comissionAmount': 2.0,
  'get': 'usdt',
  'getVol': 1030.0,
  'price': 1030.0,
  'send': 'uni',
  'sendVol': 1.0,
  'sender': 'z01x0030dfc2c4736a6ecaa2362cb65f7fb313934b33283f51caa765e3745e',
  'symbol': 'uni/usdt',
  'timestamp': 1629891728.041295,
  'tradeTxId': '5209e1a8ebe118fe1f25b27fcca9a03bca2012d81f5f40046ac99211138

In [26]:
chain = json.loads(requests.get(host+'/chain').content)['chain']
chain

[{'index': 1,
  'previousHash': '9c3b78de7eb0066418a682aec7d0e6841cd5cbe70d71ae65309d4917cd1c5681',
  'proof': 42,
  'timestamp': 1629032178.368946,
  'transactions': ['ascl,sdc,d[v,[d,vsdcaaxasx]]', 'lv,sldc,sdc ;,d;d']}]

In [38]:
sybmolSortedLst = sorted([(ordersPool[i]['sender'],ordersPool[i]['symbol'],\
                            ordersPool[i]['send'],ordersPool[i]['get'],\
                           ordersPool[i]['price'],ordersPool[i]['tradeTxId'],)\
                          for i in range(len(ordersPool))], key=itemgetter(4, 2, 1))

sybmolSortedLst

[('z01x0030dfc2c4736a6ecaa2362cb65f7fb313934b33283f51caa765e3745e',
  'btc/usdt',
  'usdt',
  'btc',
  48960.0,
  '86e46974fdd63dd773da36f3724e91ec782e62110b44637fde6322f39d4101ce'),
 ('z01x0030dfc2c4736a6ecaa2362cb65f7fb313934b33283f51caa765e3745e',
  'btc/usdt',
  'usdt',
  'btc',
  48970.0,
  'fd782f8c14c710d1e33ca276961e9cb7a61fff371ead383dac2d81ddb15af1a0'),
 ('z01x0030dfc2c4736a6ecaa2362cb65f7fb313934b33283f51caa765e3745e',
  'btc/usdt',
  'usdt',
  'btc',
  48980.0,
  '6aab946489976021faff63a985d37243a65b1ec0c3c837bc0d92c6b1154aae3f'),
 ('z01x0030dfc2c4736a6ecaa2362cb65f7fb313934b33283f51caa765e3745e',
  'btc/usdt',
  'btc',
  'usdt',
  49000.0,
  '6d348eea99de4b9a0999f1b252cdf1c28d8bba0bdecf3184c17f64291e2ab4e3'),
 ('z01x0030dfc2c4736a6ecaa2362cb65f7fb313934b33283f51caa765e3745e',
  'btc/usdt',
  'usdt',
  'btc',
  49000.0,
  '6356596306c008b19dee84b58701b0d93bdd07b17160c6caf24ce4314082b511'),
 ('z01x0030dfc2c4736a6ecaa2362cb65f7fb313934b33283f51caa765e3745e',
  'btc/usdt',
  '

In [39]:
mathchedOrders=[]
for i in range(len(sybmolSortedLst)-1):
    if (sybmolSortedLst[i][2] == sybmolSortedLst[i+1][3]) and (sybmolSortedLst[i][3] == sybmolSortedLst[i+1][2]) :

        # Split symbol and define direction and price matching
        splittedSymbol1 = re.split(r'/', sybmolSortedLst[i][1])[0]
        splittedSymbol2 = re.split(r'/', sybmolSortedLst[i+1][1])[0]

        if ((splittedSymbol1 == sybmolSortedLst[i][2] and splittedSymbol2 == sybmolSortedLst[i+1][3]) \
            and (sybmolSortedLst[i][4] <= sybmolSortedLst[i+1][4])) \
            or ((splittedSymbol1 == sybmolSortedLst[i][3] and splittedSymbol2 == sybmolSortedLst[i+1][2])\
               and (sybmolSortedLst[i][4] >= sybmolSortedLst[i+1][4])):

            mathchedOrders.append((sybmolSortedLst[i], sybmolSortedLst[i+1],))
mathchedOrders

[(('z01x0030dfc2c4736a6ecaa2362cb65f7fb313934b33283f51caa765e3745e',
   'btc/usdt',
   'btc',
   'usdt',
   49000.0,
   '6d348eea99de4b9a0999f1b252cdf1c28d8bba0bdecf3184c17f64291e2ab4e3'),
  ('z01x0030dfc2c4736a6ecaa2362cb65f7fb313934b33283f51caa765e3745e',
   'btc/usdt',
   'usdt',
   'btc',
   49000.0,
   '6356596306c008b19dee84b58701b0d93bdd07b17160c6caf24ce4314082b511'))]

In [56]:
mathchedOrders[0][1][-1]

'6356596306c008b19dee84b58701b0d93bdd07b17160c6caf24ce4314082b511'

In [ ]:
commonTxs = []
# Get matched orders
txDir = {}
for i in range(len(mathchedOrders)):
    tempLst = [mathchedOrders[i][0][0], mathchedOrders[i][1][0]]
    txTempLst=[order for order in ordersList if order['tradeTxId'] in tempLst]
    txDir[f'tx_{i}'] = txTempLst

In [270]:
txPool = requests.get(host+'/getTxPool').content
txPool

b'{"txPool":[]}\n'

In [67]:
chain = json.loads(requests.get(host+'/chain').content)['chain']
chain

[{'index': 1,
  'previousHash': '9c3b78de7eb0066418a682aec7d0e6841cd5cbe70d71ae65309d4917cd1c5681',
  'proof': 42,
  'timestamp': 1629032178.368946,
  'transactions': ['ascl,sdc,d[v,[d,vsdcaaxasx]]', 'lv,sldc,sdc ;,d;d']},
 {'index': 2,
  'previousHash': '9204ca3bc9ca2716f5a0a3ed866c9fb5b15a292c0ea53bc2e3e145b94933f393',
  'proof': 22976,
  'timestamp': 1629891791.715504,
  'transactions': [{'comissionAmount': 1.0,
    'contract': None,
    'price': 0.0,
    'recieveAmount': None,
    'recipient': '97c3a1fcdbad4085b059b22735c74faf',
    'sendAmount': 1,
    'sender': 'z01x0000000000000000000000000000000000000000000000000000000000',
    'symbol': 'zsh',
    'timestamp': 1629891791.043628,
    'tradeTxId': None},
   {'comissionAmount': 0.02,
    'contract': 'uni',
    'price': 1001.0,
    'recieveAmount': 1001.0,
    'recipient': 'z01x0030dfc2c4736a6ecaa2362cb65f7fb313934b33283f51caa765e3745e',
    'sendAmount': 1.0,
    'sender': 'z01x0030dfc2c4736a6ecaa2362cb65f7fb313934b33283f51caa765

In [ ]:
chain[0][-1][]

### NODE 1

In [71]:
# host1 = 'http://0.0.0.0:5001'
host1 = 'http://178.176.120.241:5000'

In [74]:
wallet1 = requests.post(host1+'/wallet/new', json={'password': 'lalalend'}).content
wallet1

b'{"ADDRESS":"z01x001ddb88cc07c6d9179c51b370128d07109a650cba29c5889c078e6bb0"}\n'

In [266]:
login1 = requests.post(host1+'/wallet/login', json={'password': 'lalalend'}).content
login1

b'{"ADDRESS":"z01x0011c451ec98668981d667349895f26de90940f0b55a2843eabab0ebc7","MSG":true}\n'

In [267]:
tradeTx1 = {
    'type': 'trade',
    'sender': json.loads(login1)['ADDRESS'],
    'symbol': 'btc/usdt',
    'price': 3.0,
    'send': 'btc',
    'sendVol': 2.0,
    'get': 'usdt',
    'getVol': 6.0,
    'comissionAmount':2
}

txReq1 = requests.post(host1+'/transactions/new', json=tradeTx1).content
txReq1

b'{"MSG":[3,"Tx pool synced among 1 nodes"]}\n'

In [268]:
tradeTx1 = {
    'type': 'trade',
    'sender': json.loads(login1)['ADDRESS'],
    'symbol': 'btc/usdt',
    'price': 3.0,
    'send': 'usdt',
    'sendVol': 6.0,
    'get': 'btc',
    'getVol': 2.0,
    'comissionAmount':2
}

txReq1 = requests.post(host1+'/transactions/new', json=tradeTx1).content
txReq1

b'{"MSG":[3,"Tx pool synced among 1 nodes"]}\n'

In [72]:
ordersPool1 = requests.get(host1+'/getTradeOrders').content
ordersPool1

b'{"tradeOrders":[{"comissionAmount":2.0,"get":"usdt","getVol":49010.0,"price":49010.0,"send":"btc","sendVol":1.0,"sender":"z01x0030dfc2c4736a6ecaa2362cb65f7fb313934b33283f51caa765e3745e","symbol":"btc/usdt","timestamp":1629785450.250563,"tradeTxId":"097e6811fd1bb472cf2acc15bbcf773e224b10cf4c5a814201075b48eb455a1e"},{"comissionAmount":2.0,"get":"usdt","getVol":49020.0,"price":49020.0,"send":"btc","sendVol":1.0,"sender":"z01x0030dfc2c4736a6ecaa2362cb65f7fb313934b33283f51caa765e3745e","symbol":"btc/usdt","timestamp":1629785451.023485,"tradeTxId":"c31a89faee68430c5dbc117c60d35a45080e842b591b76a80e1fe1d32376f628"},{"comissionAmount":2.0,"get":"usdt","getVol":49030.0,"price":49030.0,"send":"btc","sendVol":1.0,"sender":"z01x0030dfc2c4736a6ecaa2362cb65f7fb313934b33283f51caa765e3745e","symbol":"btc/usdt","timestamp":1629785451.748514,"tradeTxId":"ba171caf3e650036595ee8432cb0627ceb1c2c0267fb03f9ac4a0218ac2679eb"},{"comissionAmount":2.0,"get":"btc","getVol":1.0,"price":48980.0,"send":"usdt","sen

In [272]:
mineReq1 = requests.get(host1+'/mine').content
mineReq1

b'{"index":3,"message":"New Block Forged","pool_syncing":"Tx pool synced among 1 nodes","previous_hash":"49c4fa4aada5dec35ce6f03f9de1465628b9513738ce8d3dd1f18cb17d0f2a8f","proof":119240,"transactions":[{"comissionAmount":1.0,"contract":null,"price":0.0,"recieveAmount":null,"recipient":"895cc4a0852d41a9a9c4f7f0ad55d296","sendAmount":1,"sender":"z01x0000000000000000000000000000000000000000000000000000000000","symbol":"zsh","timestamp":1629723377.434202,"tradeTxId":null},{"comissionAmount":2.0,"contract":"btc","price":3.0,"recieveAmount":6.0,"recipient":"z01x0011c451ec98668981d667349895f26de90940f0b55a2843eabab0ebc7","sendAmount":2.0,"sender":"z01x0011c451ec98668981d667349895f26de90940f0b55a2843eabab0ebc7","symbol":"btc/usdt","timestamp":1629723377.440518,"tradeTxId":"fb7456550faa82f198c6026e488ce59602ae9a013ad08fa808e2b6bdff76e856"},{"comissionAmount":2.0,"contract":"usdt","price":3.0,"recieveAmount":2.0,"recipient":"z01x0011c451ec98668981d667349895f26de90940f0b55a2843eabab0ebc7","sendAm

In [177]:
txPool1 = requests.get(host1+'/getTxPool').content
txPool1

b'{"txPool":[]}\n'

In [273]:
chain1 = requests.get(host1+'/chain').content
chain1

b'{"chain":[{"index":1,"previousHash":"9c3b78de7eb0066418a682aec7d0e6841cd5cbe70d71ae65309d4917cd1c5681","proof":42,"timestamp":1629032178.368946,"transactions":["ascl,sdc,d[v,[d,vsdcaaxasx]]","lv,sldc,sdc ;,d;d"]},{"index":2,"previousHash":"9204ca3bc9ca2716f5a0a3ed866c9fb5b15a292c0ea53bc2e3e145b94933f393","proof":22976,"timestamp":1629723151.057186,"transactions":[{"comissionAmount":1.0,"contract":null,"price":0.0,"recieveAmount":null,"recipient":"d32902ea13e44d3a9f905d99620d0892","sendAmount":1,"sender":"z01x0000000000000000000000000000000000000000000000000000000000","symbol":"zsh","timestamp":1629723151.043912,"tradeTxId":null},{"comissionAmount":2.0,"contract":"btc","price":30000.0,"recieveAmount":60000.0,"recipient":"z01x00daba2c45b077691c1ada65fb07f5e740e455d4463c59028b37055900","sendAmount":2.0,"sender":"z01x00daba2c45b077691c1ada65fb07f5e740e455d4463c59028b37055900","symbol":"btc/usdt","timestamp":1629723151.057158,"tradeTxId":"8e85b87e9298b3d39259884f5edc56eb516ec865d96333783c